In [1]:
%%time
import os
import time
import sys 
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

Wall time: 2.2 s


# Example Notebook

## 1. Import the Package SmartControl

In [2]:
%%time
path = 'D:\\Repos\\PirnaCaseStudy'
sys.path.append(path) #indicate the the repository is
import SmartControl as sc

Wall time: 9.68 s


## 2. Querying data using the module *queries*

In [3]:
database_fn = 'Data/Database.db'
os.chdir(path)

#Getting the database
Get = sc.queries.Get(database_fn) #instantiates the database 

### 2.1 Fetching database status

In [4]:

VariableID = Get.VariableID('h_level') # The ID of the variable
APIDate_t , APIDate_ts = Get.APIDate(23) # The last date that the monitoring id was updated


print(VariableID)
print(f'Pandas Timestamp: {APIDate_t}')
print(f'Unix Timestamp counted from 1970 onwards: {APIDate_ts}')

0
Pandas Timestamp: 2022-12-15 14:00:00
Unix Timestamp counted from 1970 onwards: 1671112800


### 2.2 Fetching important structure data

In [4]:
%%time

MonitoringPointData_df = Get.MonitoringPointData(GageData = 1)
GageData_df = Get.GageData

Wall time: 18 ms


If one wishes to visualize the data, just type it into a cell

In [7]:
DiverData_df

,MonitoringPointID,MonitoringPointName,DiverDepth,ReferenceAltitude,WellDepth
0,23,GWM02,12.3,117.98,14.0


#### Diver Status

It returns Dataframe joining divers, wells, drills and the current status with regards to the divers' last update.

In [7]:
%%time
DiverStatus_df = Get.DiverStatus() 

Wall time: 13.4 s


In [9]:
DiverStatus_df [DiverStatus_df.MonitoringPointName.str.startswith('GW') ]

,PointID,MonitoringPointName,ReferenceAltitude,Type,TypeOfAltitude,MonitoringPointID,DiverID,DiverName,IOT,Functioning,PointsID,E,N,NextUpdate_t,NextUpdate_ts
11,26,GWM02,117.98,Well,Top Case,23,11,I-2,1,1,26,13.923774,50.965905,2022-12-15 14:00:00,1671112800
12,27,GWM03,117.88,Well,Top Case,24,12,I-3,1,1,27,13.923764,50.965858,2022-12-15 14:00:00,1671112800
13,29,GWM05,117.87,Well,Top Case,26,13,I-5,1,1,29,13.923778,50.965812,2022-12-15 14:00:00,1671112800
14,30,GWM06,117.93,Well,Top Case,27,14,I-6,1,0,30,13.923829,50.965826,2022-01-17 09:00:00,1642410000


### 2.3 Historical data

Fetching the historical values

In [4]:
%%time
# Get.LongTimeSeries(0)
# Get.ShortTimeSeries(FilterVariableID = 0, FilterMonitoringPoint = 'G01')
Get.Isolines(2022, 12, 7, 2)
# Get.HydroProfile()

Wall time: 501 ms


,MonitoringPointID,MonitoringPointName,DiverName,Time,Type,CaseTop,DiverDepth,Value,E,N
2,23,GWM02,I-2,2022-12-07 02:00:00,h_level,117.98,12.3,105.680,13.923774,50.965905
9,24,GWM03,I-3,2022-12-07 02:00:00,h_level,117.88,12.3,105.580,13.923764,50.965858
16,26,GWM05,I-5,2022-12-07 02:00:00,h_level,117.87,12.3,109.970,13.923778,50.965812
23,15,G24,124,2022-12-07 02:00:00,h_level,119.56,10.8,109.659,13.924817,50.966529
28,20,G29,125,2022-12-07 02:00:00,h_level,120.14,10.8,109.662,13.922332,50.966438
33,21,G30,126,2022-12-07 02:00:00,h_level,118.89,10.8,109.624,13.924894,50.966210


This dataframes are stored within each class as a class attribute. You can retrieve it as follows:

In [35]:
HydroProfile_df = Get.HydroProfile_df
Isolines_df = Get.Isolines_df

and so on...

Just add the suffix **'_df'** to the method name

### 2.3b - Checking up conversion

$$ Value(head) = CaseTop - DiverDepth + DiverReading(Inowas) $$

In [45]:
df.Time.astype('int64') 

2     1670536800000000000
9     1670536800000000000
16    1670536800000000000
23    1670536800000000000
28    1670536800000000000
33    1670536800000000000
Name: Time, dtype: int64

In [37]:
1670403600 1670536800'

1.6704036

In [46]:
 1670536800 / 1e9

1.6705368

In [43]:
Get.Isolines(2022, 12, 8, 22 )
df = Get.Isolines_df.copy ()
df ['DiverReading'] = np.round(df.Value - df.CaseTop + df.DiverDepth, 2)
df

,MonitoringPointID,MonitoringPointName,DiverName,Time,Type,CaseTop,DiverDepth,Value,E,N,DiverReading
2,23,GWM02,I-2,2022-12-08 22:00:00,h_level,117.98,12.3,109.840,13.923774,50.965905,4.16
9,24,GWM03,I-3,2022-12-08 22:00:00,h_level,117.88,12.3,109.730,13.923764,50.965858,4.15
16,26,GWM05,I-5,2022-12-08 22:00:00,h_level,117.87,12.3,110.150,13.923778,50.965812,4.58
23,15,G24,124,2022-12-08 22:00:00,h_level,119.56,10.8,109.846,13.924817,50.966529,1.09
28,20,G29,125,2022-12-08 22:00:00,h_level,120.14,10.8,109.834,13.922332,50.966438,0.49
33,21,G30,126,2022-12-08 22:00:00,h_level,118.89,10.8,109.811,13.924894,50.966210,1.72


## 2.4 Saving to CSV

In [31]:
os.chdir(path)

outputsname_dict = {'MonitoringPointData.csv' : MonitoringPointData_df, 'GageData.csv' : GageData_df,
                    'DiverData.csv' : DiverData_df, 'DiverStatus.csv' : DiverStatus_df,
                    'LongTimeSeries.csv' : Get.LongTimeSeries_df, 'ShortTimeSeries.csv' : Get.ShortTimeSeries_df,
                    'Isolines.csv' : Get.Isolines_df, 'HydroProfile.csv' : Get.HydroProfile_df}

def SaveDataFrames():
    folder = 'Data'
    os.chdir(path+f'\\{folder}')
    for i,j in outputsname_dict.items():
        j.to_csv(i, index = False)
        print(f'{i} saved in the {folder} folder')

if you wish to save it, uncomment and run the line below

In [32]:
SaveDataFrames()

MonitoringPointData.csv saved in the Data folder
GageData.csv saved in the Data folder
DiverData.csv saved in the Data folder
DiverStatus.csv saved in the Data folder
LongTimeSeries.csv saved in the Data folder
ShortTimeSeries.csv saved in the Data folder
Isolines.csv saved in the Data folder
HydroProfile.csv saved in the Data folder
